In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## * Objectives: To explore customer from this company for using in marketing plan.
## * Data:  UCI Online Retail II data.Between 01/12/2009 and 09/12/2011.
## * Model: RFM analysis

# Data preparation for RFM analysis.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
url='../input/online-retail-ii-uci/online_retail_II.csv'
df=pd.read_csv(url)
df

In [ ]:
df.columns

In [ ]:
country_cout_customer=df[['Country','Customer ID']].drop_duplicates()#return only choosen columns

In [ ]:
country_cout_customer

In [ ]:
country_cout_customer=country_cout_customer.groupby('Country').agg({'Customer ID':'count'}).reset_index().sort_values('Customer ID',ascending=False)

In [ ]:
# country_cout_customer=country_cout_customer.groupby(['Country'])['Customer ID'].aggregate('count').reset_index().sort_values('Customer ID',ascending=False)

In [ ]:
country_cout_customer

In [ ]:
rlt_data=df.query('Country!=("Bermuda","Hongkong")').reset_index(drop=True) #Drop these 0 Customer country

In [ ]:
rlt_data

In [ ]:
df.shape

In [ ]:
rlt_data.isnull().sum() #check null in each column

In [ ]:
df=df[pd.notnull(df['Customer ID'])] #remove null focus on the most null column that is cus ID

In [ ]:
df['Quantity'].min()# qauntity can't be negative right?

In [ ]:
df['Price'].min() # no negative

In [ ]:
#remove negative
df=df[(df['Quantity'] > 0)]

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']) #convert to datetime

In [ ]:
df.info()

In [ ]:
df['Total']=df['Quantity']*df['Price'] #caltucate total

In [ ]:
df

In [ ]:
df.shape

In [ ]:
import datetime as dt


In [ ]:
#https://pbpython.com/pandas-grouper-agg.html    agg funvtion

# **Calculate Racency,Frequency,Monetary**

In [ ]:
df['InvoiceDate'].max()

**The nearest date is '2011-12-09 12:50:00'.**

In [ ]:
#set lastet date
lasted_date = dt.datetime(2011,12,10)

In [ ]:
rfm=df.groupby('Customer ID').agg({'InvoiceDate': lambda x:(lasted_date - x.max()).days,'Invoice': lambda x: len(x),'Total': lambda x:x.sum()})

**calculate in one line of code.**

In [ ]:
rfm['InvoiceDate']=rfm['InvoiceDate'].astype(int)
rfm.rename(columns={'InvoiceDate':'Recency','Invoice':'Frequency','Total':'Monetary'},inplace=True)

Change data type and Rename Columns

In [ ]:
rfm.info()

In [ ]:
rfm.reset_index(inplace=True)

In [ ]:
rfm.head()

Finally, we've got RFM analysis.

# **Explore RFM Data**

**We're going to use *Descriptive statistics* for RFM data from above.**

###  **1. Recency**

In [ ]:
rfm['Recency'].describe()

Average number of Recency is 200 days. overall customers look low activity? 200 days is more than half year?

## we should look at Distribution.

In [ ]:
import seaborn as sns
from scipy.stats import skewnorm
x=rfm['Recency']
ax=sns.displot(x,kind="hist",kde=True)

We've got right-skew here. It looks good (the lower Recency the better it is!!)  but there are accumulation around 400 days.

### **2.Frequency**

In [ ]:
rfm['Frequency'].describe()

some customer buy more than 10,000 transaction.

In [ ]:
import seaborn as sns
x=rfm['Frequency']
ax=sns.displot(x,kind="hist",kde=True)

Data has got long tail.so I use some treshold to cut and visualize it.

In [ ]:
x=rfm.query('Frequency<1500')['Frequency']
ax=sns.displot(x,kde=True,height=6, aspect=2)

Frequency has right skew distribution.

### **3. Monetary**

In [ ]:
rfm['Monetary'].describe()

some customer paid more than 600,000(big big outlier). Average paid per customer is 3,017.

In [ ]:
x=rfm['Monetary']
ax=sns.displot(x,kde=True,height=5,aspect=2,bins=120)

In [ ]:
x=rfm.query('Monetary < 10000')['Monetary']
ax=sns.displot(x,kde=True,height=6,aspect=2)

Right skew again.

# **Clustering**

I use quantile to cluster our customer.

In [ ]:
quantiles = rfm[['Recency','Frequency','Monetary']].quantile(q=[0.25,0.5,0.75])
quantiles=quantiles.to_dict()

In [ ]:
quantiles

Create Function for clustering segment.(rank from 1- 4 ; 1 is excellent , 4 is bad .)

In [ ]:
def recency(x,y,z):
    if x <=  y[z][0.25]:
        return 1
    elif x <= y[z][0.5]:
        return 2
    elif x <= y[z][0.75]:
        return 3
    else:
        return 4
def fandm(x,y,z):
    if x <=  y[z][0.25]:
        return 4
    elif x <= y[z][0.5]:
        return 3
    elif x <= y[z][0.75]:
        return 2
    else:
        return 1

apply to all rfm data

In [ ]:
rfm['R']=rfm['Recency'].apply(recency,args=(quantiles,'Recency'))
rfm['F']=rfm['Frequency'].apply(fandm,args=(quantiles,'Frequency'))
rfm['M']=rfm['Monetary'].apply(fandm,args=(quantiles,'Monetary'))

In [ ]:
rfm.head()

# **Ranking**

In [ ]:
rfm['Score']=rfm['R']+rfm['F']+rfm['M']
rfm['Gruop_score']=rfm['R'].map(str)+rfm['F'].map(str)+rfm['M'].map(str)

sum R,F,M to Score

In [ ]:
rfm

In [ ]:
loyalty_level=['platinum','gold','silver','bronze']
score_cut=pd.qcut(rfm['Score'],q=4,labels=loyalty_level)


Ranking Score in each loyalty level i.e. 'platinum','gold','silver','bronze'.

In [ ]:
rfm['loyalty_level']=score_cut

# **Results**

Explore each 'loyalty_level' 

In [ ]:
dg=rfm.groupby('loyalty_level').agg({'loyalty_level':'count'})

In [ ]:
dg

count each loyalty_level

In [ ]:
dg.plot(kind='bar');

* **This is what online retail is facing. platinum and gold customer would be OK.**
* **But silver and bronze should be focused,Espeacially bronze.Becuase it can turn to be Churn customer(Or now it is.)**
* **Marketing plan should be immediatly lunch to these customer such as special offer,discount,etc.**